In [69]:
import numpy as np
import pandas as pd

#prepare data
df_adjacency = pd.read_csv('task-2-adjacency_matrix.csv')

#set a huge value to impossible connections
df_adjacency[df_adjacency == '-'] = 10000

#prepare data
df_tickets = pd.read_csv('task-2-nodes.csv', header=None)

In [70]:
(df_adjacency.iloc[:, 0] == df_tickets.iloc[:, 0]).all()

True

In [71]:
places = df_adjacency.iloc[:, 0]

In [72]:
df_adjacency_drop_first_column = df_adjacency.iloc[:, 1:]

for col in df_adjacency_drop_first_column.columns:
  df_adjacency_drop_first_column[col] = pd.to_numeric(df_adjacency_drop_first_column[col])
adjacency_mx = df_adjacency_drop_first_column.to_numpy()

tickets = pd.to_numeric(df_tickets.iloc[:, 1]).to_numpy()

C:\Users\lenovo\AppData\Local\Temp\ipykernel_9596\2290946303.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_adjacency_drop_first_column[col] = pd.to_numeric(df_adjacency_drop_first_column[col])


In [73]:
places[2]

'Вокзал'

In [74]:
"""Capacited Vehicles Routing Problem (CVRP)."""

from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp


def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data["distance_matrix"] = adjacency_mx
    data["demands"] = tickets
    data["vehicle_capacities"] = [10]*15
    data["num_vehicles"] = 15
    data["depot"] = 2
    return data


solution_data = []

def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    print(f"Objective: {solution.ObjectiveValue()}")
    total_distance = 0
    total_load = 0
    for vehicle_id in range(data["num_vehicles"]):
        vehicle_data = []
        index = routing.Start(vehicle_id)
        plan_output = f"Route for vehicle {vehicle_id}:\n"
        route_distance = 0
        route_load = 0
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route_load += data["demands"][node_index]
            plan_output += f" {node_index} Load({route_load}) -> "
            previous_index = index
            vehicle_data.append(places[node_index])
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id
            )
        #vehicle_data.append(places[manager.IndexToNode(index)])
        plan_output += f" {manager.IndexToNode(index)} Load({route_load})\n"
        plan_output += f"Distance of the route: {route_distance}m\n"
        plan_output += f"Load of the route: {route_load}\n"
        print(plan_output)
        total_distance += route_distance
        total_load += route_load
        solution_data.append(vehicle_data)
    print(f"Total distance of all routes: {total_distance}m")
    print(f"Total load of all routes: {total_load}")


def main():
    """Solve the CVRP problem."""
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(
        len(data["distance_matrix"]), data["num_vehicles"], data["depot"]
    )

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)

    # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data["distance_matrix"][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Add Capacity constraint.
    def demand_callback(from_index):
        """Returns the demand of the node."""
        # Convert from routing variable Index to demands NodeIndex.
        from_node = manager.IndexToNode(from_index)
        return data["demands"][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data["vehicle_capacities"],  # vehicle maximum capacities
        True,  # start cumul to zero
        "Capacity",
    )

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
    )
    search_parameters.local_search_metaheuristic = (
        routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH
    )
    search_parameters.time_limit.FromSeconds(1)

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        print_solution(data, manager, routing, solution)


if __name__ == "__main__":
    main()

Objective: 250183
Route for vehicle 0:
 2 Load(0) ->  21 Load(5) ->  20 Load(8) ->  25 Load(10) ->  2 Load(10)
Distance of the route: 14m
Load of the route: 10

Route for vehicle 1:
 2 Load(0) ->  38 Load(5) ->  32 Load(8) ->  10 Load(8) ->  44 Load(10) ->  2 Load(10)
Distance of the route: 20012m
Load of the route: 10

Route for vehicle 2:
 2 Load(0) ->  1 Load(0) ->  19 Load(9) ->  9 Load(9) ->  2 Load(9)
Distance of the route: 20m
Load of the route: 9

Route for vehicle 3:
 2 Load(0) ->  50 Load(0) ->  49 Load(0) ->  17 Load(5) ->  51 Load(5) ->  18 Load(7) ->  47 Load(10) ->  2 Load(10)
Distance of the route: 10029m
Load of the route: 10

Route for vehicle 4:
 2 Load(0) ->  46 Load(9) ->  2 Load(9)
Distance of the route: 20000m
Load of the route: 9

Route for vehicle 5:
 2 Load(0) ->  55 Load(0) ->  26 Load(9) ->  2 Load(9)
Distance of the route: 20003m
Load of the route: 9

Route for vehicle 6:
 2 Load(0) ->  14 Load(2) ->  35 Load(4) ->  0 Load(4) ->  29 Load(7) ->  39 Load(10) -

In [75]:
max_len = max(max([len(path) for path in solution_data]), 16)
max_len

16

In [76]:
for i, path in enumerate(solution_data):
    while len(solution_data[i]) < max_len:
        solution_data[i].append('Вокзал')

In [77]:
solution_data

[['Вокзал',
  'Нижегородская ярмарка',
  'Собор Александра Невского',
  'Спасский Староярмарочный собор',
  'Вокзал',
  'Вокзал',
  'Вокзал',
  'Вокзал',
  'Вокзал',
  'Вокзал',
  'Вокзал',
  'Вокзал',
  'Вокзал',
  'Вокзал',
  'Вокзал',
  'Вокзал'],
 ['Вокзал',
  'Площадь Минина и Пожарского',
  'Усадьба Рукавишниковых',
  'Паромная переправа 1',
  'Литературный музей им. Горького',
  'Вокзал',
  'Вокзал',
  'Вокзал',
  'Вокзал',
  'Вокзал',
  'Вокзал',
  'Вокзал',
  'Вокзал',
  'Вокзал',
  'Вокзал',
  'Вокзал'],
 ['Вокзал',
  'Канавинский мост 2',
  'Стрелка',
  'Борский мост 2',
  'Вокзал',
  'Вокзал',
  'Вокзал',
  'Вокзал',
  'Вокзал',
  'Вокзал',
  'Вокзал',
  'Вокзал',
  'Вокзал',
  'Вокзал',
  'Вокзал',
  'Вокзал'],
 ['Вокзал',
  'Перекресток 3',
  'Перекресток 2',
  'Зоопарк',
  'Перекресток 4',
  'Копосовская дубрава',
  'Святой источник Пафнутия Балахнинского',
  'Вокзал',
  'Вокзал',
  'Вокзал',
  'Вокзал',
  'Вокзал',
  'Вокзал',
  'Вокзал',
  'Вокзал',
  'Вокзал'],
 ['Вок

In [78]:
import csv
with open('solution.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerows(solution_data)